# 第二章 OpenAI 函数调用 OpenAI Function Calling

本章主要介绍 OpenAI 的 API 提供的新功能，在最近几个月，OpenAI 对一些新的模型进行了微调，分别是`gpt-3.5-turbo-0613`和`gpt-4-0613`。

假如我们有一个工具函数，在一些特定情况下需要调用，经过微调后，这些模型能够传入新的参数，可以通过这个新的参数来自动判断是否调用工具函数，如果判断需要调用工具函数，会返回这个工具函数和对应的输入参数。


- [一、OpenAI函数新参数](#一openai函数新参数)
  - [1.1 简单例子：得到当前天气](#11-简单例子得到当前天气)  
  - [1.2 新参数：functions](#12-新参数functions) 
  - [1.3 相关提示调用结果](#13-相关提示调用结果)    
  - [1.4 无关提示调用结果](#14-无关提示调用结果)
- [二、Function Call参数模式](#二function-call参数模式)
  - [2.1 自动判断是否调用](#21-自动判断是否调用)    
  - [2.2 强制不调用](#22-强制不调用)    
  - [2.3 强制调用](#23-强制调用)  
- [三、函数调用以及执行函数](#三函数调用以及执行函数)   
- [四、英文版提示](#四英文版提示)

## 一、OpenAI函数新参数

首先我们直接定义使用`OPENAI_API_KEY`，方便后续去调用OpenAI的API接口，并利用其函数。

In [1]:
import os
import openai

os.environ['OPENAI_API_KEY'] = "YOUR_API_KEY"
openai.api_key = os.environ['OPENAI_API_KEY']

### 1.1 简单例子：得到当前天气

首先我们使用 OpenAI 的使用的第一个例子，定义了一个`get_current_weather`的函数，正常来说，获取当前天气是语言模型本身不能完全做到的事情。因此，我们希望能够将语言模型和这样的函数结合起来，以当前的信息来增强它。

在这个函数，我们固定了返回的值，比如温度固定为 22 摄氏度，但在实际应用中，这可能涉及到调用天气   API 或一些外部知识源。

In [28]:
import json

# 定义一个函数，用于获取给定位置的当前天气
def get_current_weather(location, unit="摄氏度"):
    """获取指定位置的当前天气"""
    # 模拟返回相同的天气情况的示例函数
    # 在实际应用环境中，这可以是天气API
    # 创建一个天气信息的字典
    weather_info = {
        "location": location,  # 天气的位置
        "temperature": "22",  # 温度
        "unit": unit,  # 温度单位，默认为摄氏度
        "forecast": ["晴", "多云"],  # 天气预报
    }
    # 将天气信息转换为JSON格式的字符串并返回
    return json.dumps(weather_info)

### 1.2 新参数：functions

那我们如何将这样的函数传给语言模型呢？OpenAI引入了一个名为`functions`的新参数，通过该参数，我们可以传递一个函数定义列表。由于我们只有一个函数，所以列表中只有一个元素。这是一个JSON对象，具有几个不同的参数。

- name：函数的名称
- description：函数的描述
- parameters：参数对象，里面有一些属性设置
  - type：类型
  - properties：本身是一个对象，传入的是对应的参数的描述。在例子中我们可以看到有两个元素，`location`和`unit`。每个这些元素都有一个类型，即字符串，然后是一个描述。`unit`是一个外部参数的设置，比如这里我们希望它可以是摄氏度或华氏度，所以我们可以在这里定义他的类型和以及枚举参数值。
  - required：必填的参数，比如这里我们需要的参数就是`location`。

在函数定义中，`description`以及在`properties`中的参数非常重要，因为这些将直接传递给语言模型，语言模型将使用这些描述来判断是否使用此函数。

In [29]:
# 定义一个函数
functions = [
    {
        "name": "get_current_weather",
        "description": "获取指定位置的当前天气情况",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "城市和省份，例如：北京，北京市",
                },
                "unit": {"type": "string", "enum": ["摄氏度", "华氏度"]},
            },
            "required": ["location"],
        },
    }
]

### 1.3 相关提示调用结果

接下来我们就可以定义一个有关于天气的问题，如”北京的天气怎么样？“，然后使用 OpenAI 的函数进行调用对话的 API。首先我们得选取较新的模型，如`gpt-3.5-turbo-0613`，然后我们再将上述定义的        function 函数传入，查看最后的响应结果。

从结果上来看，返回的消息的角色是助手，内容为空，而是有一个函数调用参数`function_call`，其中包含两个对象，`name`和`arguments`。`Name`是`get_current_weather`，这与我们传递的函数的名称相同，然后`arguments`是这个 JSON 格式的字典，里面是我们需要的参数。

In [ ]:
import openai

# 定义输入消息
messages = [
    {
        "role": "user",
        "content": "北京的天气怎么样?"
    }
]

# 调用OpenAI的ChatCompletion API获取响应
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions # 传入function参数
)

# 打印响应结果
print(response)

{
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "function_call": {
          "arguments": "{\n  \"location\": \"\u5317\u4eac\uff0c\u5317\u4eac\u5e02\",\n  \"unit\": \"\u6444\u6c0f\u5ea6\"\n}",
          "name": "get_current_weather"
        },
        "role": "assistant"
      }
    }
  ],
  "created": 1709602992,
  "id": "chatcmpl-8zE7kIlpxzjsiuxH4q6wtRh8CLDH3",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 30,
    "prompt_tokens": 95,
    "total_tokens": 125
  }
}


我们也可以看到响应的参数，比如在这里就是两个参数，分别是`location`和`unit`。

In [ ]:
# 打印传入参数
print(response["choices"][0]["message"]["function_call"]["arguments"])

{
  "location": "北京，北京市",
  "unit": "摄氏度"
}


如果我们仔细看响应的话，我们会发现内容现在是空的，`function_call`是一个字典，`function_call`中的`arguments`参数本身也是一个 JSON 字典。因此我们可以使用`json.loads`将其加载到 Python 字典中。它返回的参数可以直接传递给我们上述定义的`get_current_weather`函数。

我们会发现，使用 OpenAI 进行函数调用并不直接调用工具函数，我们还是需要去调用工具函数，它只是告诉我们要调用哪个函数，即名称，以及该函数的参数应该是什么。以及由于他并没有去执行函数，所以说，如果我们在使用`json.loads`解码的时候遇到了一些问题，那实际上是模型的问题，所以这一部分可以考虑在写工具函数的时候做一些保护措施，后续也会讨论这一个点。

In [ ]:
# 获取response里面的message信息
response_message = response["choices"][0]["message"]

print(response_message)

{
  "content": null,
  "function_call": {
    "arguments": "{\n  \"location\": \"\u5317\u4eac\uff0c\u5317\u4eac\u5e02\",\n  \"unit\": \"\u6444\u6c0f\u5ea6\"\n}",
    "name": "get_current_weather"
  },
  "role": "assistant"
}


In [ ]:
# 打印参数
print(response["choices"][0]["message"]["function_call"]["arguments"])

{
  "location": "北京，北京市",
  "unit": "摄氏度"
}


In [ ]:
# 将JSON格式的字符串转换为Python对象
args = json.loads(response_message["function_call"]["arguments"])

# 调用get_current_weather函数并传入参数args
get_current_weather(args)

'{"location": {"location": "\\u5317\\u4eac\\uff0c\\u5317\\u4eac\\u5e02", "unit": "\\u6444\\u6c0f\\u5ea6"}, "temperature": "22", "unit": "\\u6444\\u6c0f\\u5ea6", "forecast": ["\\u6674", "\\u591a\\u4e91"]}'

### 1.4 无关提示调用结果

接下来，我们也探讨一下，如果问的问题与函数无关会产生什么样的，也就是与天气无关，会返回什么样的信息呢？从结果上来看，我们可以返回的内容正常并且没有`function_call`参数，也就是在语言模型中判断不使用工具函数。

In [ ]:
# 与天气无关提示调用
messages = [
    {
        "role": "user",
        "content": "你好!",
    }
]

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",
)

print(response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "\u4f60\u597d\uff01\u6709\u4ec0\u4e48\u6211\u53ef\u4ee5\u5e2e\u52a9\u4f60\u7684\u5417\uff1f",
        "role": "assistant"
      }
    }
  ],
  "created": 1709602993,
  "id": "chatcmpl-8zE7lZ3keV5JGVTFqvAWOGxit4cXj",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 19,
    "prompt_tokens": 88,
    "total_tokens": 107
  }
}


由于编码问题，我们可以单独打印对应的返回的文本信息

In [ ]:
print(response["choices"][0]["message"]["content"])

你好！有什么我可以帮助你的吗？


## 二、Function Call参数模式

`function_call`参数一共有3种模式，我们可以传递其他参数`function_call`以强制模型使用或不使用函数。

1. 默认情况下，它设置为`auto`，也就是模型自行选择。
2. 在第二种模式中，我们可以强制它调用一个函数。如果我们希望始终返回函数
3. 另一种模式是`none`。这会强制语言模型不使用提供的任何函数。

### 2.1 自动判断是否调用

`auto`模式就是大模型自行选择是否要返回参数，这一部分也是默认的，上述所有的方式都是`auto`的模式

### 2.2 强制不调用

`none`模式对于这个例子来说并不重要，因为内容`你好`不需要函数调用，所以我们看到它没有被使用。

In [9]:
# 无关提示强制不调用
messages = [
    {
        "role": "user",
        "content": "你好",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none", # 传入参数强制不调用
)
print(response)
print(response["choices"][0]["message"]["content"])

{
  "id": "chatcmpl-8nfuv5tlp6UaSwUfmeHppJkQyFpCW",
  "object": "chat.completion",
  "created": 1706849893,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "\u4f60\u597d\uff01\u6709\u4ec0\u4e48\u53ef\u4ee5\u5e2e\u52a9\u4f60\u7684\u5417\uff1f"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 88,
    "completion_tokens": 17,
    "total_tokens": 105
  },
  "system_fingerprint": null
}
你好！有什么可以帮助你的吗？


那如果我们在需要使用工具的函数时候强制不调用（也就是使用第二种模式）会出现什么结果呢？从结果上来看，他依然有正常的`role`和`content`，但是因为我们强制不调用函数，所以他在试图返回正确的参数，但是并不正确。

In [10]:
# 相关提示强制不调用
messages = [
    {
        "role": "user",
        "content": "北京天气怎么样?",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none", # 传入参数强制不调用
)
print(response)
print(response["choices"][0]["message"]["content"])

{
  "id": "chatcmpl-8nfv0wFX6tqcaoU2gT4KqICymXDxa",
  "object": "chat.completion",
  "created": 1706849898,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "\u8bf7\u7a0d\u7b49\uff0c\u6211\u4e3a\u60a8\u67e5\u627e\u5317\u4eac\u7684\u5929\u6c14\u60c5\u51b5\u3002"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 95,
    "completion_tokens": 18,
    "total_tokens": 113
  },
  "system_fingerprint": null
}
请稍等，我为您查找北京的天气情况。


### 2.3 强制调用

`function_call`参数的最后一个模式是强制调用函数，方法也是很简单，只要在参数里面传入这个函数的名字即可。在这一部分，我们指定`name`为`get_current_weather`，这将强制它使用`get_current_weather`函数。如果我们查看结果，我们实际上会看到返回了这个`function_call`对象，其中有`name`为`get_current_weather`的一些参数。

为了好玩，这里面传了一个于天气无关的参数，然后强制他使用`get_current_weather`函数，但我们传入的参数中绝对没有任何信息关于它应该如何调用该函数。所以在这里，他编造了北京，北京市的参数，如果我们再次运行它，它会不断地调用北京市的参数。

我们还可以尝试使用不同的模型函数来调用，不同的参数值，不同的输入消息，但是这里面要注意一个事情，首先，函数本身和描述都会计入传递给OpenAI的令牌使用限制。所以如果我们运行这个，我们可以看到返回的提示令牌是95。如果我们注释掉`functions`和`function_call`，我们可以看到提示令牌减少到15。因为OpenAI模型对令牌有限制，因此，在构造要传递给OpenAI的消息时，现在不仅需要注意消息的长度，还需要注意传递的函数的长度。

In [13]:
# 无关提示强制调用函数
messages = [
    {
        "role": "user",
        "content": "你好!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"}, # 强制调用函数get_current_weather
)
print(response)
print(response["choices"][0]["message"]["function_call"]["arguments"])

{
  "id": "chatcmpl-8nfvkQDayf7kmYNdx4G84dkl1L5YC",
  "object": "chat.completion",
  "created": 1706849944,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"\u5317\u4eac\uff0c\u5317\u4eac\u5e02\"\n}"
        }
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 95,
    "completion_tokens": 12,
    "total_tokens": 107
  },
  "system_fingerprint": null
}
{
  "location": "北京，北京市"
}


## 三、函数调用以及执行函数

最后，让我们看一下如何将这些函数调用和实际执行函数调用的结果传递回语言模型。这很重要，因为通常我们希望使用语言模型确定要调用的函数，然后运行该函数，但然后将其传递回语言模型以获得最终响应。

我们会经过这样的流程，首先我们问问题得到一个带有`function call`参数的响应，然后我们将这个消息添加到我们的消息列表中。然后我们可以模拟使用语言模型提供的参数调用`get_current_weather`函数，并且保存到一个变量`observation`中。接着我们定义一个新的消息列表，表示刚刚调用函数的结果，这里面有一个重要的点就是`role`等于`function`，也就是告诉语言模型这是调用函数的响应。除此之外，还传递函数的名称`name`以及`content`变量，设为上述计算的`observation`。

如果我们然后使用这个消息列表调用语言模型，我们可以看到语言模型回答的非常好：北京的天气目前是22摄氏度，天气以晴天为主，也有多云的情况。

In [ ]:
# 与天气无关提示调用
messages = [
    {
        "role": "user",
        "content": "你好!",
    }
]

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",
)

print(response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "\u4f60\u597d\uff01\u6709\u4ec0\u4e48\u6211\u53ef\u4ee5\u5e2e\u52a9\u4f60\u7684\u5417\uff1f",
        "role": "assistant"
      }
    }
  ],
  "created": 1709602993,
  "id": "chatcmpl-8zE7lZ3keV5JGVTFqvAWOGxit4cXj",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 19,
    "prompt_tokens": 88,
    "total_tokens": 107
  }
}


由于编码问题，我们可以单独答应对应的返回的文本信息

In [ ]:
print(response["choices"][0]["message"]["content"])

你好！有什么我可以帮助你的吗？


## 四、英文版提示

**1.1 简单例子：得到当前天气**

In [17]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

**1.2 定义function函数**

In [18]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

**1.3 相关提示调用结果**

In [19]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)

print(response)

{
  "id": "chatcmpl-8nfyFSmPyo4RoV79D5Urk1QzLCf1C",
  "object": "chat.completion",
  "created": 1706850099,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }
      },
      "logprobs": null,
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 18,
    "total_tokens": 100
  },
  "system_fingerprint": null
}


**1.4 无关提示调用结果**

In [20]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
)

**2.1 自动判断是否调用**

自动判断是否调用函数，判断提示是否和函数相关

In [21]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",
)
print(response)

{
  "id": "chatcmpl-8nfyHfC32MVE2gpE6HZxtmC7f7Mvi",
  "object": "chat.completion",
  "created": 1706850101,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 76,
    "completion_tokens": 10,
    "total_tokens": 86
  },
  "system_fingerprint": null
}


**2.2 强制不调用**

无关提示强制不调用，结果正常

In [22]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

{
  "id": "chatcmpl-8nfyfrOoacEQCPO3O2sYLukAdbxX4",
  "object": "chat.completion",
  "created": 1706850125,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 77,
    "completion_tokens": 9,
    "total_tokens": 86
  },
  "system_fingerprint": null
}


**2.3 强制调用**

强制无关问题的调用

In [24]:

messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

{
  "id": "chatcmpl-8nfyzFFeUyzJDYVbdiVDgPEtNrnKE",
  "object": "chat.completion",
  "created": 1706850145,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"San Francisco, CA\"\n}"
        }
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 83,
    "completion_tokens": 12,
    "total_tokens": 95
  },
  "system_fingerprint": null
}


强制有关问题调用，结果和`auto`一样

In [26]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

{
  "id": "chatcmpl-8nfzav6WbYIDLPHXYwaBXTM0byxaj",
  "object": "chat.completion",
  "created": 1706850182,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 89,
    "completion_tokens": 11,
    "total_tokens": 100
  },
  "system_fingerprint": null
}
